# Python-Notebook zum Senden von Lastschrift-Einzugsnachrichten

## Setup

In [196]:
import pathlib

ROOT_DIR_PATH = pathlib.Path("../")
DATA_DIR_PATH = ROOT_DIR_PATH / "data" / "Einzugsnachricht"
TEMPLATE_DIR_PATH = ROOT_DIR_PATH / "templates"

In [197]:
from dotenv import load_dotenv

load_dotenv(dotenv_path=(ROOT_DIR_PATH / ".env"))

True

In [198]:
from datetime import datetime, timedelta


# Set the value date to the last day of the previous year
VALUE_DATE = datetime.now().replace(month=1, day=1) - timedelta(days=1)

# Set the update deadline to 14 days from now
UPDATE_DEADLINE = datetime.now() + timedelta(days=14)

# Display the value date
VALUE_DATE.strftime("%d.%m.%Y")

'31.12.2022'

## Import der Mitglieder, Beitröge und Lastschriftmandate

In [199]:
MEMBER_FILE_PATH = DATA_DIR_PATH / 'Mitglieder.xlsx'

In [200]:
import warnings
import pandas

def read_excel(
    workbook_path: str,
    sheet_name: str,
    header_map: dict = {},
    skip_rows: int = 0
) -> pandas.DataFrame:
    warnings.filterwarnings('ignore', module='openpyxl')
    df = pandas.read_excel(
        workbook_path,
        sheet_name=sheet_name,
        skiprows=max(0, skip_rows - 1),
    )
    df = df.drop(columns=[col for col in df if col not in header_map.keys()])
    df = df.rename(columns=header_map)
    warnings.filterwarnings('default', module='openpyxl')
    return df

In [201]:
# Load the members form the "Mitglieder" worksheet
members = read_excel(
    workbook_path=MEMBER_FILE_PATH,
    sheet_name="Mitglieder",
    header_map={
        'ID': 'id',
        'Anrede': 'salutation',
        'Vorname': 'first_name',
        'Nachname': 'last_name',
        'E-Mail': 'email',
        'Status': 'status',
        'Mitgliedschaft': 'member_type'
    },
    skip_rows=4
)

# Load the payment information from the "Finanzen" worksheet
payment_info = read_excel(
    workbook_path=MEMBER_FILE_PATH,
    sheet_name="Finanzen",
    header_map={
        'MitgliedsNr.': 'member_id',
        'Beitrag': 'amount_fee',
        'Spende': 'amount_donation',
        'Gesamt': 'amount_total',
        'Voller Name': 'account_holder',
        'Referenz': 'mandate_reference',
        'Gläubiger-ID': 'creditor_id',
        'Erteilt Am': 'issue_date',
        'IBAN (Anonymisiert)': 'iban_anonymized',
        'BIC (Anonymisiert)': 'bic_anonymized',
        'Kreditinstitut': 'credit_institute'
    },
    skip_rows=4
)
# Convert NaN to zero in amount fields and convert the mandate reference to a string
payment_info['amount_donation'] = payment_info['amount_donation'].fillna(0)
payment_info['amount_fee'] = payment_info['amount_fee'].fillna(0)
payment_info['amount_total'] = payment_info['amount_total'].fillna(0)
payment_info['mandate_reference'] = payment_info['mandate_reference'].fillna(0)
payment_info['mandate_reference'] = payment_info['mandate_reference'].astype(int).astype(str)

# Filter members that are still part of the JuBO, i.e. have a status of "Aktiv",
# "Passiv" or "Inaktiv" and combine them with the payment information
paying_members = members[members["status"].isin(["Aktiv", "Passiv", "Inaktiv"])]

payments = paying_members.merge(
    payment_info,
    how='left',
    left_on='id',
    right_on='member_id'
)
payments

,id,salutation,first_name,last_name,email,status,member_type,member_id,amount_fee,amount_donation,amount_total,account_holder,credit_institute,iban_anonymized,bic_anonymized,creditor_id,mandate_reference,issue_date
0,10000,Herr,Gökhan,Kiral,gkiral@gmx.net,Inaktiv,Erwachsen,10000,50.0,0.0,50.0,Gökhan Kiral,Scharnhauser Bank,DE1XXXXXXXXXXXXXXXX008,GENODEXXXXX,DE13JUB00000934427,10000,2014-03-22
1,10001,Herr,Tobias,Gralke,tobiasgralke@posteo.de,Inaktiv,Erwachsen,10001,50.0,0.0,50.0,Tobias Gralke,GLS Gemeinschaftsbank Bochum,DE4XXXXXXXXXXXXXXXX200,GENODEXXXXX,DE13JUB00000934427,10001,2014-02-01
2,10002,Frau,Lena,Fritz,l.enafritz@gmx.de,Inaktiv,Erwachsen,10002,50.0,0.0,50.0,Lena Fritz,Kreissparkasse Esslingen-Nürtingen,DE3XXXXXXXXXXXXXXXX828,ESSLDEXXXXX,DE13JUB00000934427,10002,2014-03-30
3,10003,Herr,Manuel,Woletz,mwoletz@online.de,Passiv,Erwachsen,10003,50.0,0.0,50.0,Manuel Woletz,Sparda-Bank Baden-Württemberg,DE9XXXXXXXXXXXXXXXX836,GENODEXXXXX,DE13JUB00000934427,10003,2010-05-01
4,10004,Frau,Hannah,Bauer,bauer-hannah@hotmail.de,Passiv,Erwachsen,10004,50.0,0.0,50.0,Hannah Bauer,Volksbank Esslingen,DE8XXXXXXXXXXXXXXXX000,GENODEXXXXX,DE13JUB00000934427,10004,2014-03-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,10115,Frau,Naima,Albertz,naima@salbertz.de,Aktiv,Ermäßigt,10115,30.0,100.0,130.0,Sandra Albertz,Volksbank Welzheim,DE4XXXXXXXXXXXXXXXX003,GENODEXXXXX,DE13JUB00000934427,10115,2022-09-22
60,10116,Herr,Mohammad,Skirek,skirekm@gmail.com,Aktiv,Ermäßigt,10116,30.0,0.0,30.0,Zeid Skirek,Kreissparkasse Esslingen-Nürtingen,DE6XXXXXXXXXXXXXXXX733,ESSLDEXXXXX,DE13JUB00000934427,10116,2022-10-15
61,10117,Herr,Joschka Cosmo,Basset,joschkabasset@gmail.com,Aktiv,Ermäßigt,10117,30.0,0.0,30.0,Florian Basset,Sparda-Bank Baden-Württemberg,DE3XXXXXXXXXXXXXXXX976,GENODEXXXXX,DE13JUB00000934427,10117,2022-09-26
62,10118,Frau,Kira,Szikszai,k.szikszui@gmx.de,Aktiv,Ermäßigt,10118,30.0,0.0,30.0,,NaN,NaN,NaN,DE13JUB00000934427,10118,NaT


In [202]:
# Filter members that are missing data
print('Checking for missing data.')
missing_data_payments = payments[payments['issue_date'].isna()]
payments = payments[payments['issue_date'].notna()]
if len(missing_data_payments) > 0:
    print(f"Found {len(missing_data_payments)} members with missing data:")
    missing_data_payments
else:
    print("No missing data found.")

Checking for missing data.
Found 2 members with missing data:


In [203]:
# Display the total amounts
print(f'Number of paying members: {len(payments)}/{len(members)}')
print(f'Fees:\t\t{payments["amount_fee"].sum()}€')
print(f'Donations:\t{payments["amount_donation"].sum()}€')
print(f'Total:\t\t{payments["amount_total"].sum()}€')

Number of paying members: 62/91
Fees:		2260.0€
Donations:	400.0€
Total:		2660.0€


## Vorbereiten der E-Mails

In [204]:
import os

TEMPLATE_NAME = 'einzugsnachricht.html.jinja'

EMAIL_USER = os.getenv('EMAIL_USER')
EMAIL_SIGNATURE_NAME  = os.getenv('EMAIL_SIGNATURE_NAME')
EMAIL_SIGNATURE_ROLE  = os.getenv('EMAIL_SIGNATURE_ROLE')
EMAIL_SIGNATURE_EMAIL = os.getenv('EMAIL_SIGNATURE_EMAIL')
EMAIL_SIGNATURE_PHONE = os.getenv('EMAIL_SIGNATURE_PHONE')

In [205]:
from dataclasses import dataclass
from email.message import EmailMessage
from typing import List, Optional


@dataclass
class Email:
    sender: str
    to: List[str]
    subject: str
    content: str
    cc: Optional[List[str]] = None
    bcc: Optional[List[str]] = None

    def as_message(self) -> EmailMessage:
        message = EmailMessage()
        message["From"] = self.sender
        message["To"] = ", ".join(self.to)
        if self.cc:
            message["Cc"] = ", ".join(self.cc)
        if self.bcc:
            message["Bcc"] = ", ".join(self.bcc)
        message["Subject"] = self.subject
        message.set_content(self.content, subtype="html")
        return message

    def __str__(self) -> str:
        return self.as_message().as_string()

In [206]:
from jinja2 import Environment, FileSystemLoader, Template


def read_template(name: str) -> Template:
    env = Environment(
        loader=FileSystemLoader(TEMPLATE_DIR_PATH), trim_blocks=True, lstrip_blocks=True
    )
    env.globals["format_date"] = lambda x: x.strftime("%d.%m.%Y")
    env.globals["format_currency"] = lambda x: "{:,.2f}€".format(x).replace(".", ",")
    return env.get_template(name)


template = read_template(TEMPLATE_NAME)

In [207]:
from IPython.display import display, HTML

# Prepare the e-mails
emails: List[Email] = []
for payment in payments.itertuples():
    content = template.render(
        salutation=payment.salutation,
        first_name=payment.first_name,
        member_type=payment.member_type,
        amount_fee=payment.amount_fee,
        amount_donation=payment.amount_donation,
        amount_total=payment.amount_total,
        account_holder=payment.account_holder,
        iban_anonymized=payment.iban_anonymized,
        bic_anonymized=payment.bic_anonymized,
        mandate_reference=payment.mandate_reference,
        creditor_id=payment.creditor_id,
        issue_date=payment.issue_date,
        value_date=VALUE_DATE,
        contact_email=EMAIL_USER,
        update_deadline=UPDATE_DEADLINE,
        signature_name=EMAIL_SIGNATURE_NAME,
        signature_role=EMAIL_SIGNATURE_ROLE,
        signature_email=EMAIL_SIGNATURE_EMAIL,
        signature_phone=EMAIL_SIGNATURE_PHONE,
    )
    email = Email(
        sender=EMAIL_USER,
        to=[payment.email],
            subject=f'JuBO e.V. | Mitgliedsbeitrag { VALUE_DATE.year } | Mitglied Nr. M{ payment.id } { payment.first_name } { payment.last_name }',
        content=content
    )
    emails.append(email)

emails[0]

Email(sender='presse@jubo.info', to=['gkiral@gmx.net'], subject='JuBO e.V. | Mitgliedsbeitrag 2022 | Mitglied Nr. M10000 Gökhan Kiral', content='<html>\n    <head>\n        <meta charset="utf-8">\n\n    <style>\n        body {\n            font-family: Arial, Helvetica, sans-serif;\n        }\n\n        table {\n            text-align: left;\n            border-collapse: collapse;\n        }\n\n        #bill-details tr {\n            border-bottom: 1px solid black;\n        }\n\n        #bill-details td,\n        #bill-details th {\n            padding: 10px;\n        }\n\n        #bill-details tr.total-row {\n            border: none;\n        }\n\n        #bill-details tr.total-row td,\n        tr.total-row th {\n            font-weight: bold;\n        }\n\n        #bank-details td {\n            padding-bottom: 5px;\n            padding-right: 10px;\n        }\n    </style>\n    </head>\n    <body>\n    <p>\n            Lieber\n Gökhan,\n    </p>\n    <p>\n        wie jedes Jahr um 

In [208]:
display(HTML(emails[0].content))

## Erstellen der E-Mails im Postfach

In [209]:
import os

EMAIL_USER      = os.getenv('EMAIL_USER')
EMAIL_PASSWORD  = os.getenv('EMAIL_PASSWORD')
EMAIL_IMAP_HOST = os.getenv('EMAIL_IMAP_HOST')
EMAIL_IMAP_PORT = os.getenv('EMAIL_IMAP_PORT')
EMAIL_SMTP_HOST = os.getenv('EMAIL_SMTP_HOST')
EMAIL_SMTP_PORT = os.getenv('EMAIL_SMTP_PORT')

EMAIL_SEND_TRIES = 3

In [210]:
import imaplib
import smtplib
import ssl
import time


class EmailClientWithSSL:
    _imap = imaplib.IMAP4_SSL
    _smtp = smtplib.SMTP_SSL

    def __init__(
        self,
        user: str,
        password: str,
        imap_host: str,
        imap_port: int,
        smtp_host: str,
        smtp_port: int,
    ) -> None:
        # Connect to the IMAP server with SSL
        try:
            self._imap = imaplib.IMAP4_SSL(imap_host, imap_port)
            self._imap.login(user, password)
        except Exception as ex:
            self.close()
            raise ex
        # Connect to the SMTP server with SSL
        try:
            self._smtp = smtplib.SMTP_SSL(
                smtp_host, smtp_port, context=ssl.create_default_context()
            )
            self._smtp.login(user, password)
        except:
            self.close()
            raise ex

    def send(self, email: Email) -> None:
        self._smtp.sendmail(
            from_addr=email.sender, to_addrs=email.to, msg=str(email)
        )
        self._imap.append(
            mailbox="Sent",
            flags="\\Seen",
            date_time=imaplib.Time2Internaldate(time.time()),
            message=str(email).encode("utf8"),
        )

    def draft(self, email: Email) -> None:
        self._imap.append(
            mailbox="Drafts",
            flags="",
            date_time=imaplib.Time2Internaldate(time.time()),
            message=str(email).encode("utf8"),
        )

    def close(self) -> None:
        if self._imap:
            try:
                self._imap.close()
            except:
                pass
        if self._smtp:
            try:
                self._smtp.close()
            except:
                pass


email_client = EmailClientWithSSL(
    user=EMAIL_USER,
    password=EMAIL_PASSWORD,
    imap_host=EMAIL_IMAP_HOST,
    imap_port=EMAIL_IMAP_PORT,
    smtp_host=EMAIL_SMTP_HOST,
    smtp_port=EMAIL_SMTP_PORT,
)

In [214]:
# This flag is used to prevent accidentally creating e-mails. Set it to True
# before running the cell to create the e-mail drafts.
SEND_EMAILS = True

# Create the drafts
if SEND_EMAILS:
    for email in emails[:1]:
        print(f'Creating e-mail draft for "{",".join(email.to)}"')
        for t in range(EMAIL_SEND_TRIES):
            try:
                email_client.draft(email)
                break
            except Exception as ex:
                print(f'Failed to create draft. Retrying ({t+1}/{EMAIL_SEND_TRIES}) after {t * 5} seconds...')
                time.sleep(t * 5)
    print(f'{len(emails)} e-mail drafts created successfully.')
else:
    print(f'Not creating e-mail drafts. Set the "SEND_EMAILS" flag to True to create the e-mail drafts.')

Creating e-mail draft for "gkiral@gmx.net"
62 e-mail drafts created successfully.
